<a href="https://colab.research.google.com/github/tanya12181/Machine-Learning-Internship-at-Internity/blob/main/Copy_of_Preprocessing_for_major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
"""
The fit_on_texts method is a part of Keras tokenizer class which is used to update 
the internal vocabulary for the texts list. 
We need to call be before using other methods of texts_to_sequences or texts_to_matrix.

fit_on_texts.word_index assigns a unique index for each word.
"""
import json 
import re
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


def pad_sequences(sequences,maxlen,padding = 'pre',value=0):
  padding_sequences = sequences
  if padding=='pre':
    for seq in padding_sequences:
      while(len(seq)<maxlen):
        seq.insert(0,value)
  else:
    for seq in padding_sequences:
      while(len(seq)<maxlen):
        seq.append(value)
  return np.array(padding_sequences)





def texts_to_sequences(training_sentences,word_index):
  seq=[]
  bad_char = [',','?','/','_','@','#','*']
  for sentence in training_sentences:
    w=[]
    for c in bad_char:
      sentence = sentence.replace(c,'')
    for word in sentence.split():
      word = word.lower()
      w.append(word_index[word])
    seq.append(w)
  return seq



class label_encoder:

  def __init__(self,training_labels):
    self.training_labels = training_labels
    self.class_array = []
    self.class_labels = []


  def find_class_array(self):
    self.class_array = []
    self.class_labels = []
    idx = 0
    for data in self.training_labels:
      if data not in self.class_array:
        self.class_array.append(data)
        self.class_labels.append(idx)
        idx+=1
    self.class_array = sorted(self.class_array)
    #print(self.class_array)
    

  def Label_Encoder(self):
    num_array = []
    #print(self.class_array)
    d = {self.class_array[i]:self.class_labels[i] for i in range(len(self.class_array))}
    for label in self.training_labels:
      num_array.append(d[label])
    return(np.array(num_array))


  def inverse_transform(similarity_vector,self):
    max_index = None
    max = 0
    for index in range(len(similarity_vector)):
      if max<similarity_vector[index]:
        max_index = index
        max = similarity_vector[index]
    return self.class_array[max_index]
    

In [13]:
import nltk
nltk.download('stopwords', quiet = True)
from nltk.corpus import stopwords

stop_words = list(stopwords.words('english'))
#stop_words


In [14]:
def remove_stopwords(stop_words,training_sentences):
  new_sentences = []
  for sentence in training_sentences:
    s = ''
    words = sentence.split()
    for word in words:
      if word not in stop_words:
        s+=(word+' ')
    s=s.strip()
    new_sentences.append(s)
  return new_sentences



In [16]:
with open('intents.json') as file:
    data = json.load(file)


training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])




#removing stopwords
training_sentences = remove_stopwords(stop_words,training_sentences)


print("after removing stopwords:")
print("new sentences : ",training_sentences)

#encoding training labels
lbl_encoder = label_encoder(training_labels) 
lbl_encoder.find_class_array()
training_labels = lbl_encoder.Label_Encoder()


print("after encoding : ",training_labels)


vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) 
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = texts_to_sequences(training_sentences,word_index)
padded_sequences = pad_sequences(sequences,max_len)

epochs = 550
#history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

after removing stopwords:
new sentences :  ['Hi', 'How', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'Bye', 'See later', 'Goodbye', 'Nice chatting you, bye', 'Till next time', 'Thanks', 'Thank', "That's helpful", 'Awesome, thanks', 'Thanks helping', 'How could help me?', 'What do?', 'What help provide?', 'How helpful?', 'What support offered', 'How check Adverse drug reaction?', 'Open adverse drugs module', 'Give list drugs causing adverse behavior', 'List drugs suitable patient adverse reaction', 'Which drugs dont adverse reaction?', 'Open blood pressure module', 'Task related blood pressure', 'Blood pressure data entry', 'I want log blood pressure results', 'Blood pressure data management', 'I want search blood pressure result history', 'Blood pressure patient', 'Load patient blood pressure result', 'Show blood pressure results patient', 'Find blood pressure results ID', 'Find pharmacy', 'Find pharmacy', 'List pharmacies nearby', 'Locate pharmacy', 'Search pharmacy', 'Loo

In [17]:
print(word_index)
word_index = dict(sorted(word_index.items()))
#word_index

{'<OOV>': 1, 'blood': 2, 'pressure': 3, 'patient': 4, 'adverse': 5, 'hospital': 6, 'how': 7, 'drugs': 8, 'pharmacy': 9, 'thanks': 10, 'what': 11, 'reaction': 12, 'list': 13, 'data': 14, 'i': 15, 'want': 16, 'results': 17, 'search': 18, 'find': 19, 'bye': 20, 'helpful': 21, 'help': 22, 'open': 23, 'module': 24, 'result': 25, 'lookup': 26, 'hi': 27, 'is': 28, 'anyone': 29, 'there': 30, 'hey': 31, 'hola': 32, 'hello': 33, 'good': 34, 'day': 35, 'see': 36, 'later': 37, 'goodbye': 38, 'nice': 39, 'chatting': 40, 'you': 41, 'till': 42, 'next': 43, 'time': 44, 'thank': 45, "that's": 46, 'awesome': 47, 'helping': 48, 'could': 49, 'me': 50, 'do': 51, 'provide': 52, 'support': 53, 'offered': 54, 'check': 55, 'drug': 56, 'give': 57, 'causing': 58, 'behavior': 59, 'suitable': 60, 'which': 61, 'dont': 62, 'task': 63, 'related': 64, 'entry': 65, 'log': 66, 'management': 67, 'history': 68, 'load': 69, 'show': 70, 'id': 71, 'pharmacies': 72, 'nearby': 73, 'locate': 74, 'searching': 75, 'transfer': 76,

In [18]:
seq = texts_to_sequences(training_sentences,word_index)
print(seq)

[[27], [7], [28, 29, 30], [31], [32], [33], [34, 35], [20], [36, 37], [38], [39, 40, 41, 20], [42, 43, 44], [10], [45], [46, 21], [47, 10], [10, 48], [7, 49, 22, 50], [11, 51], [11, 22, 52], [7, 21], [11, 53, 54], [7, 55, 5, 56, 12], [23, 5, 8, 24], [57, 13, 8, 58, 5, 59], [13, 8, 60, 4, 5, 12], [61, 8, 62, 5, 12], [23, 2, 3, 24], [63, 64, 2, 3], [2, 3, 14, 65], [15, 16, 66, 2, 3, 17], [2, 3, 14, 67], [15, 16, 18, 2, 3, 25, 68], [2, 3, 4], [69, 4, 2, 3, 25], [70, 2, 3, 17, 4], [19, 2, 3, 17, 71], [19, 9], [19, 9], [13, 72, 73], [74, 9], [18, 9], [26, 6], [75, 6, 76, 4], [15, 16, 18, 6, 14], [6, 26, 4], [77, 6, 78]]


In [19]:
print(padded_sequences)
#padded_sequences2[0,:].shape
# if seq == sequences:
#   print('true')

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 27]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  7]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 28 29 30]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 31]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 32]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 33]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 34 35]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 20]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 36 37]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 39 40 41 20]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 42 43 44]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 45]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 46 21]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0